In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import display, HTML

import marshaltools

In [2]:
import requests
import re

def check_status(code):
    if code != 200:
        if code in marshaltools.gci_utils.httpErrors.keys():
            raise ValueError(marshaltools.gci_utils,httpErrors[code])
        else:
            raise ValueError('Error %i'%code)

def iniatiate_transfer(name, to_program=32):
    # Get sourceid
    r = requests.post('http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi', 
                      auth=(pl1.user, pl1.passwd), 
                      data={'name': name})
    check_status(r.status_code)            
    sourceid = re.findall('[0-9]+', re.findall('transfer_source.cgi\?sourceid=[0-9]+', r.text)[0])[0]
    
    # id of from_program
    r = requests.post('http://skipper.caltech.edu:8080/cgi-bin/growth/transfer_source.cgi', 
                  auth=(pl1.user, pl1.passwd), 
                  data={'sourceid': sourceid})
    check_status(r.status_code)            
    programid = re.findall('[0-9]+', re.findall('From Program: <select name=from_program >\n<option value=\"[0-9]+\">', r.text)[0])[0]

    # create transfer
    r = requests.post('http://skipper.caltech.edu:8080/cgi-bin/growth/transfer_source.cgi', 
                  auth=(pl1.user, pl1.passwd), 
                  data={'commit': 'yes', 'sourceid': sourceid, 'id': -1, 'from_program': programid,
                        'to_program': 32, 'share': 'yes', 'comment': '', 'status': 'Submit'})
    check_status(r.status_code)            
    
    return sourceid

def confirm_transfer(sourceids):
    # get currently pending transactions to determine transaction id
    r = requests.post('http://skipper.caltech.edu:8080/cgi-bin/growth/pending_transactions.cgi', 
                      auth=(pl1.user, pl1.passwd))
    check_status(r.status_code)    
    
    for name, sourceid in sourceids.items():
        pattern = '<input type=hidden name="id" value=[0-9]+>\s*<input type=hidden name="sourceid" value=%s>'%sourceid
        query = re.findall(pattern, r.text)
        
        if len(query) > 0:
            transid = re.findall('[0-9]+', query[0])[0]
    
            r2 = requests.post('http://skipper.caltech.edu:8080/cgi-bin/growth/transfer_source.cgi', 
                              auth=(pl1.user, pl1.passwd), 
                              data={'commit': 'yes', 'sourceid': sourceid, 'id': transid, 'status': 'Accept'})
        else:
            print('%s (id %s) not found in transactions.'%(name, sourceid))
        

In [3]:
# Start by loading the list of SNe for the Cosmology program
pl1 = marshaltools.ProgramList('Cosmology')

INFO:marshaltools.ProgramList:Initialized ProgramList for program Cosmology (ID 1)
INFO:marshaltools.ProgramList:Loaded 1501 saved sources for program Cosmology.


In [4]:
pls = {}

In [ ]:
# Furthermore load a list of all source on the marshal that you have access to.
# This may take a while.
for p in pl1.program_list:
    if p['name'] != 'Cosmology' and p['name'] not in pls.keys():
        pls[p['name']] = marshaltools.ProgramList(p['name'], timeout=300) 

INFO:marshaltools.ProgramList:Initialized ProgramList for program ZTF Science Validation (ID 0)
INFO:marshaltools.ProgramList:Loaded 3136 saved sources for program ZTF Science Validation.
INFO:marshaltools.ProgramList:Initialized ProgramList for program Redshift Completeness Factor (ID 2)
INFO:marshaltools.ProgramList:Loaded 3534 saved sources for program Redshift Completeness Factor.
INFO:marshaltools.ProgramList:Initialized ProgramList for program Gravitational Lenses (ID 3)
INFO:marshaltools.ProgramList:Loaded 87 saved sources for program Gravitational Lenses.
INFO:marshaltools.ProgramList:Initialized ProgramList for program Stripped Envelope Supernovae (ID 4)
INFO:marshaltools.ProgramList:Loaded 213 saved sources for program Stripped Envelope Supernovae.
INFO:marshaltools.ProgramList:Initialized ProgramList for program Infant Supernovae (ID 5)
INFO:marshaltools.ProgramList:Loaded 900 saved sources for program Infant Supernovae.
INFO:marshaltools.ProgramList:Initialized ProgramList 

In [ ]:
# Figure out which SNe should be added to the Cosmology program
new = []
for pl_ in pls.values():
    for s_ in pl_.sources.values():
        if (s_['name'] not in pl1.sources.keys() 
            and (s_['name'], s_['classification']) not in new
            and s_['classification'] is not None 
            and s_['classification'].startswith('SN Ia')):
            new.append((s_['name'], s_['classification']))
            
print('Number of SNe Ia currently in the Cosmology program:', 
      len([a for a in pl1.sources.values() 
      if a['classification'] is not None and
      a['classification'].startswith('SN Ia')]))
print('Number of SNe Ia to be added:', len(new))

In [12]:
# Add them in groups of 10, i.e. 10 SNe are added to the transfer page and then confirmed.
# This saves a few HTML requests. Do not set n to a much higher value because this will add
# a lot of SNe to the transfer page, making it hard to use for a few minutes, best to avoid that.

n = 10

while len(new) > 0: 
    sourceids = {}
    for k in range(min(len(new), n)):
        name = new.pop(0)[0]
        print(name)
    
        sourceids[name] = iniatiate_transfer(name)

    confirm_transfer(sourceids)
    print('Done.')

ZTF19abpbqor
ZTF18acrfklz
ZTF19aboxjlj
ZTF19aafmfxg
ZTF19abkcpxe
ZTF19aaphkxx
ZTF19aargqez
ZTF19aawgolt
ZTF19aayuxpq
ZTF19aaynsws
Done.
ZTF19abangub
ZTF19abcfzdy
ZTF19abddqhy
ZTF19aazpnlc
ZTF19abdyawn
ZTF19abpveqn
ZTF19abeytzv
ZTF19abfpupe
ZTF19aayxlsg
ZTF19abgfrbl
Done.
ZTF19abfwnar
ZTF19abewqfb
ZTF19abgbkwv
ZTF19abhxllw
ZTF19abgqwed
ZTF19abjpkdz
ZTF19abjibet
ZTF19ablekwo
ZTF19abnxglw
ZTF19abowcic
Done.
ZTF19abnkrcd
ZTF19abhzdjp
ZTF19abkdehm
ZTF19abpbmli
ZTF19aavnzti
ZTF19aawyepw
ZTF19aazpvyr
ZTF19aaypehm
ZTF19abafrvy
ZTF19abasxkg
Done.
ZTF19abanfrn
ZTF19abcejlc
ZTF19abatvit
ZTF19abdjsss
ZTF19abdkcts
ZTF19abdkdoz
ZTF19abcejoa
ZTF19abdkfqe
ZTF19abdduix
ZTF19abegxob
Done.
ZTF19abdkgns
ZTF19abegruk
ZTF19abdqaao
ZTF19abdgpuv
ZTF19abetjcx
ZTF19abdjwtv
ZTF19abeloei
ZTF19abdkbwy
ZTF19abfqhis
ZTF19abctnpk
Done.
ZTF19abcncdg
ZTF18abdcudb
ZTF19abfvemi
ZTF19abdsntm
ZTF19abfvovo
ZTF19abexfad
ZTF19abfaqqo
ZTF19abfvhlx
ZTF19abfjjlk
ZTF19abdoznh
Done.
ZTF19abgprpq
ZTF19abfvpkq
ZTF19abgsyrp
ZTF18abug

In [66]:
print('Number of SNe Ia now in the Cosmology program:', 
      len([a for a in pl1.sources.values() 
      if a['classification'] is not None and
      a['classification'].startswith('SN Ia')]))